In [1]:
from spark_utils import get_spark_session
import pyspark.sql.functions as F

spark = get_spark_session()
# run transform_latest.py if this don't exist
test_data = spark.read.parquet('data_transformed/amex-default-prediction/test_data_aggregated')
train_data = spark.read.parquet('data_transformed/amex-default-prediction/train_data_aggregated')
# run format_data.py if these don't exist
train_labels = spark.read.parquet('data/amex-default-prediction/train_labels')
sample_submission = spark.read.parquet('data/amex-default-prediction/sample_submission')

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/20 23:07:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/06/20 23:07:28 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/06/20 23:07:28 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
', '.join(train_data.columns)

'customer_ID, S_2_first, S_2_last, num_statements, P_2_first, P_2_last, P_2_mean, D_39_first, D_39_last, D_39_mean, B_1_first, B_1_last, B_1_mean, B_2_first, B_2_last, B_2_mean, R_1_first, R_1_last, R_1_mean, S_3_first, S_3_last, S_3_mean, D_41_first, D_41_last, D_41_mean, B_3_first, B_3_last, B_3_mean, D_42_first, D_42_last, D_42_mean, D_43_first, D_43_last, D_43_mean, D_44_first, D_44_last, D_44_mean, B_4_first, B_4_last, B_4_mean, D_45_first, D_45_last, D_45_mean, B_5_first, B_5_last, B_5_mean, R_2_first, R_2_last, R_2_mean, D_46_first, D_46_last, D_46_mean, D_47_first, D_47_last, D_47_mean, D_48_first, D_48_last, D_48_mean, D_49_first, D_49_last, D_49_mean, B_6_first, B_6_last, B_6_mean, B_7_first, B_7_last, B_7_mean, B_8_first, B_8_last, B_8_mean, D_50_first, D_50_last, D_50_mean, D_51_first, D_51_last, D_51_mean, B_9_first, B_9_last, B_9_mean, R_3_first, R_3_last, R_3_mean, D_52_first, D_52_last, D_52_mean, P_3_first, P_3_last, P_3_mean, B_10_first, B_10_last, B_10_mean, D_53_fir

In [3]:
%%time
from format_data import CATEGORICAL_VARIABLES
from encoder import CategoricalToIntegerEncoders, HolidayFeatures, FourierFeatures

categorical_cols = []
for c in CATEGORICAL_VARIABLES:
    categorical_cols += [
        f'{c}_first',
        f'{c}_last',
        f'{c}_mode',
    ]

encs = CategoricalToIntegerEncoders(columns=categorical_cols).fit(train_data)


years = [row['S_2_first'].year for row in train_data.select('S_2_first').distinct().collect()]
years += [row['S_2_first'].year for row in test_data.select('S_2_first').distinct().collect()]
years += [row['S_2_last'].year for row in train_data.select('S_2_last').distinct().collect()]
years += [row['S_2_last'].year for row in test_data.select('S_2_last').distinct().collect()]
years = sorted(list(set(years)))
print(years)
years = [min(years) + i for i in range(-1, max(years) - min(years) + 2)]
print(years)

# holiday_first = HolidayFeatures(column='S_2_first', years=years)
holiday_last = HolidayFeatures(column='S_2_last', years=years)


# fourier_first = FourierFeatures(
#     column='S_2_first',
#     hour_of_day=False,
#     minute_of_hour=False,
#     use_period_fraction=.5,
# )
fourier_last = FourierFeatures(
    column='S_2_last',
    # last dates are in a single month
    week_of_year=False,
    # no sub-date level information
    hour_of_day=False,
    minute_of_hour=False,
    use_period_fraction=.5,
)

[2017, 2018, 2019]
[2016, 2017, 2018, 2019, 2020]
CPU times: user 403 ms, sys: 67.2 ms, total: 470 ms
Wall time: 21.7 s


In [4]:
%%time
from format_data import TARGET_VARIABLE, DATE_VARIABLES, ID_VARIABLES

# make train_pdf
train_pdf = train_data.join(train_labels, on='customer_ID', how='inner')
train_pdf = fourier_last.transform(spark=spark, df=train_pdf)
train_pdf = holiday_last.transform(spark=spark, df=train_pdf)
train_pdf = encs.transform(spark=spark, df=train_pdf).toPandas()

# make test_pdf
test_pdf = fourier_last.transform(spark=spark, df=test_data)
test_pdf = holiday_last.transform(spark=spark, df=test_pdf)
test_pdf = encs.transform(spark=spark, df=test_pdf).toPandas()

non_feature_columns = [
    TARGET_VARIABLE,
    *ID_VARIABLES,
    *DATE_VARIABLES.keys(),
    'S_2_first',
    'S_2_last',
]
feature_columns = [c for c in train_pdf.columns if c not in non_feature_columns]
print(f'len(feature_columns): {len(feature_columns)}\n', ', '.join(feature_columns))

22/06/20 23:08:02 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


len(feature_columns): 661
 num_statements, P_2_first, P_2_last, P_2_mean, D_39_first, D_39_last, D_39_mean, B_1_first, B_1_last, B_1_mean, B_2_first, B_2_last, B_2_mean, R_1_first, R_1_last, R_1_mean, S_3_first, S_3_last, S_3_mean, D_41_first, D_41_last, D_41_mean, B_3_first, B_3_last, B_3_mean, D_42_first, D_42_last, D_42_mean, D_43_first, D_43_last, D_43_mean, D_44_first, D_44_last, D_44_mean, B_4_first, B_4_last, B_4_mean, D_45_first, D_45_last, D_45_mean, B_5_first, B_5_last, B_5_mean, R_2_first, R_2_last, R_2_mean, D_46_first, D_46_last, D_46_mean, D_47_first, D_47_last, D_47_mean, D_48_first, D_48_last, D_48_mean, D_49_first, D_49_last, D_49_mean, B_6_first, B_6_last, B_6_mean, B_7_first, B_7_last, B_7_mean, B_8_first, B_8_last, B_8_mean, D_50_first, D_50_last, D_50_mean, D_51_first, D_51_last, D_51_mean, B_9_first, B_9_last, B_9_mean, R_3_first, R_3_last, R_3_mean, D_52_first, D_52_last, D_52_mean, P_3_first, P_3_last, P_3_mean, B_10_first, B_10_last, B_10_mean, D_53_first, D_53

In [5]:
import numpy as np

X_fit = train_pdf[feature_columns].reset_index(drop=True)
X_test = test_pdf[feature_columns].reset_index(drop=True)
print(X_fit.shape, X_test.shape)

y_fit = np.array(train_pdf[TARGET_VARIABLE])
print(np.unique(y_fit, return_counts=True))

(458913, 661) (924621, 661)
(array([0., 1.], dtype=float32), array([340085, 118828]))


In [6]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X_fit, y_fit)
print(X_train.shape, X_valid.shape, y_train.shape, y_valid.shape)

(344184, 661) (114729, 661) (344184,) (114729,)


In [7]:
%%time
import mlflow
from lightgbm import LGBMClassifier
from evaluation import feval_amex, feval_amex_gini, feval_amex_top4

mlflow.lightgbm.autolog()
experiment_id = mlflow.get_experiment_by_name('v2_agg+season.ipynb').experiment_id
with mlflow.start_run(experiment_id=experiment_id) as run:
    run_id = run.info.run_id
    m = LGBMClassifier().fit(
        X=X_train, y=y_train, categorical_feature=encs.columns_encoded,
        eval_set=[(X_valid, y_valid)],
        eval_metric=['auc', 'average_precision', feval_amex, feval_amex_gini, feval_amex_top4],
    )

/usr/local/lib/python3.8/dist-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/usr/local/lib/python3.8/dist-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['B_30_first_CategoricalToIntegerEncoder', 'B_30_last_CategoricalToIntegerEncoder', 'B_30_mode_CategoricalToIntegerEncoder', 'B_38_first_CategoricalToIntegerEncoder', 'B_38_last_CategoricalToIntegerEncoder', 'B_38_mode_CategoricalToIntegerEncoder', 'D_114_first_CategoricalToIntegerEncoder', 'D_114_last_CategoricalToIntegerEncoder', 'D_114_mode_CategoricalToIntegerEncoder', 'D_116_first_CategoricalToIntegerEncoder', 'D_116_last_CategoricalToIntegerEncoder', 'D_116_mode_CategoricalToIntegerEncoder', 'D_117_first_CategoricalToIntegerEncoder', 'D_117_last_CategoricalToIntegerEncoder', 'D_117_mode_CategoricalToIntegerEncoder', 'D_120_first_CategoricalToIntegerEncoder', 'D_120_l

[1]	valid_0's auc: 0.933419	valid_0's average_precision: 0.818307	valid_0's binary_logloss: 0.520426	valid_0's amex: 0.69659	valid_0's amex_gini: 0.867245	valid_0's amex_top4: 0.525936
[2]	valid_0's auc: 0.937503	valid_0's average_precision: 0.838594	valid_0's binary_logloss: 0.480513	valid_0's amex: 0.71237	valid_0's amex_gini: 0.875101	valid_0's amex_top4: 0.549639
[3]	valid_0's auc: 0.940183	valid_0's average_precision: 0.84382	valid_0's binary_logloss: 0.448297	valid_0's amex: 0.719411	valid_0's amex_gini: 0.880554	valid_0's amex_top4: 0.558268
[4]	valid_0's auc: 0.941533	valid_0's average_precision: 0.849449	valid_0's binary_logloss: 0.421644	valid_0's amex: 0.723213	valid_0's amex_gini: 0.883157	valid_0's amex_top4: 0.56327
[5]	valid_0's auc: 0.942875	valid_0's average_precision: 0.852568	valid_0's binary_logloss: 0.399306	valid_0's amex: 0.726942	valid_0's amex_gini: 0.885879	valid_0's amex_top4: 0.568004
[6]	valid_0's auc: 0.943734	valid_0's average_precision: 0.854883	valid_0'

2022/06/20 23:12:49 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/usr/local/lib/python3.8/dist-packages/mlflow/models/signature.py:129: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."


CPU times: user 5min 37s, sys: 8.84 s, total: 5min 46s
Wall time: 1min 45s


In [8]:
from format_data import PREDICTION_VARIABLE
import pandas as pd

pred_test = pd.DataFrame({
    'customer_ID': test_pdf['customer_ID'],
    PREDICTION_VARIABLE: m.predict(X_test, raw_score=True),
})
pred_test.head()

: 

: 

In [ ]:
# pred_test.to_csv(f'{run_id}.csv', index=False)

In [ ]:
# !kaggle competitions submit -c amex-default-prediction -f 0261ff0b99024adc88cd4b8566e2a4d1.csv -m "experiments/7/runs/0261ff0b99024adc88cd4b8566e2a4d1"